# 3D Visualization of a Static Neutron Star

This notebook walks you through some of the visualization you can do with a static neutron star with cell-centred data. This is an IPython notebook, so as you read through it, evaluate the code with `shift+enter`. As we go, I'll also make suggestions about other things you can do or try.

This tutorial only scratches the surface of what yt can do. A great resource is the yt documentation, which you can find [here.](http://yt-project.org/docs/dev/)

## Creating a file

First we need to create a file to explore. We convert the native carpet format to a `SimulationIO` file. You only need to do this once.

In [ ]:
import subprocess
subprocess.call('/usr/local/bin/sio-convert-carpet-output '
                +'./static-tov-cell.s5 '
                +'./data/static_tov_cell/output-0000/'
                +'static_tov_cell_centred/*.h5',
               shell=True)

## Importing yt and preparing the notebook

To use yt to visualize the data we need to import it. yt is just a python package, so that's easy. 

We also need to tell the IPython notebook to present plots inline. We can do that with `%matplotlib inline`. 

In [ ]:
import yt
%matplotlib inline

## Loading the data

yt is pretty smart. If we give it a file name, it automatically figures out what kind of file it is and reads in the data correctly.

But for `SimulationIO` files, we also need to specify the `configuration`. yt reads in only one timestep at a time, so the configuration specifies the time step and time level (some refinement levels take more time steps than others).

In the future, there will be a cleaner interface, but for now you must submit the configuration as a string.

yt prints out a lot of debugging output when it loads a file. You can either read it, or you can turn it off with the following settings in `~/.yt/config` (see [here](http://yt-project.org/doc/faq/index.html) for an explanation):
```bash
[yt]
loglevel = 40
```

You can also make the output a little easier to read by clicking to the left of the output cell in the notebook. This makes it fold into a submenu which you can scroll through separately.

In [ ]:
fpath = "./static-tov-cell.s5"
configurationname='iteration.0000000000-timelevel.0'
ds = yt.load(fpath,
             configuration=configurationname)

The fundamental object in yt is a `dataset` object, which is returned when you call `yt.load`. It's typically called `ds`. We'll pass the `dataset` object into lots of functions.

### Exercise

How would you load vertex-centred data instead? Or a dataset missing missing things like grid cell locations? (Hint: look at [my slides](http://52.28.232.90/jmiller/simulationio-yt-slides.pdf).) You will need to use a different data set for this. I recomend a new IPython notebook.

### Exercise

How would you iterate through several timesteps of data? This dataset only has one timestep, so you'll need a different one.

## Examining the Data

yt is capable of a fair amount of data analysis by itself. For instance, we can ask it about the grid in the dataset we just loaded.

In [ ]:
ds.print_stats()

So our grid has five refinement levels, each $60^3$ cells. yt's default units are `cgs` units, but it can handle many different unit types. The units yt assumes for the hydrodynamical variables are the defaults in `HYDROBASE` in Cactus. You can find the details [here](https://einsteintoolkit.org/documentation/ThornDoc/EinsteinBase/HydroBase/).

We can also ask for things like how wide the domain is. It tells us in units of "code length", which is basically whatever arbitrary units you specified in your parameter file.

In [ ]:
print ds.domain_width

Same with domain left and right edge:

In [ ]:
print ds.domain_left_edge
print ds.domain_right_edge

Or even the left and right edges of every single independent discretization block. yt calls separate discretization blocks "grids." These correspond to anything that Carpet has split up into a separate zone, including a refinement region, a component (i.e., a separate memory domain), etc.

To access yt's grids, we use the `index` class, which contains the grids in a data set.

In [ ]:
for grid in ds.index.grids:
    print grid.LeftEdge,grid.RightEdge

There's actually syntactic sugar for this:

In [ ]:
ds.index.grid_left_edge

### Fields 

yt treats each variable as a `field`. We can see what fields are "native" to the file by asking yt for a "field list":

In [ ]:
ds.field_list

But yt has many more "derived fields" which are automatically calculated from these quantities. Derived fields also include convenient names for the native simulationio fields. For instance, the name `density` maps to `HYDROBASE::rho`. Let's look at the list of derived fields. (Warning, it's a VERY long list.)

In [ ]:
ds.derived_field_list

By convention, all continuum variables are prefixed with `gas`. Don't worry. This won't come up anywhere but here. The user interface mostly surpresses that.

### Accessing field data

If you want, you can directly inspect the field data. Just access the grid you want (as above) and then index into it as if it were a Python dictionary. That'll give you a 3D array containing the data for that field in that grid.

In [ ]:
grid0 = ds.index.grids[0]
density0 = grid0['density']
print density0.shape

### Exercise

See if you can define a new derived field, say the linear momentum. Use [this](http://yt-project.org/docs/dev/quickstart/derived_fields_and_profiles.html?highlight=derived%20field) web page to guide you.

## Submanifolds

We can also look only at specific regions of a dataset. For example, a sphere centred at the origin:

In [ ]:
sp = ds.sphere([0.0, 0.0, 0.0], (50, 'code_length'))

This generates a `sphere` object, which we can interact with in a number of ways and which can give us several physical quantities.

In [ ]:
print sp

In [ ]:
print sp.quantities.keys()

In [ ]:
print sp.quantities.total_quantity('ones')

In [ ]:
print sp.quantities.extrema('lapse')

In [ ]:
print sp.quantities.total_quantity('density')

## Simple Visualization

Okay! The moment you've all been waiting for! Let's make some plots!

### Slice Plots

A slice plot just takes a 2D slice of 3D data along a coordinate axis. You call `yt.SlicePlot` and feed in the dataset, the axis you want to ignore, and the variable you want to plot. Let's plot the $xx$-component of the metric.

In [ ]:
p = yt.SlicePlot(ds, "z", "metric_xx")
p.show()

That's not very detailed! Can we make it easier to look at? Let's set the colorbar to linear scale instead of log, zoom in a bit, and add annotations telling us where the grids are.

In [ ]:
p = yt.SlicePlot(ds, "z", "metric_xx")
p.set_cmap(field="all", cmap="jet")
p.set_log('metric_xx',False)
p.annotate_grids()
p.set_width(100,'km')
p.show()

That's better! We can also save the plot with 

```python
p.save('static-tov-slice.png')
```

Try it if you want.

### Exercise

yt also let's you perform slice plots "off-axis." Look through the [yt documentation](http://yt-project.org/docs/dev/index.html) and figure out how. Should the plot look any different? If so you how?

### Plotting the grid level

It can sometimes be convenient to plot refinement level locations. You can do that with the special `grid_level` field.

In [ ]:
p = yt.SlicePlot(ds, 'z', "grid_level")
p.set_log("grid_level",False)
p.annotate_grids()
p.show()

### Projection Plots

A projection plot is very literally a plot of a mathematical projection onto a plane. yt integrates the whole domain along a direction and plots the resulting data. Let's integrate the lapse along the z axis. (We can also set the width in the plot call.)

In [ ]:
prj = yt.ProjectionPlot(ds, 'z','lapse',width=200)
prj.set_cmap(field="all", cmap="jet")
prj.set_log('lapse',False)
prj.show()

### Exercise

Just as there are off-axis slice plots. There are off-axis projection plots. See if you can figure out how to do it.

### Volume renders

And now, finally, let's do a volume render! Let's render the density. yt has a very cool "scene" interface for volume rendering that lets you navigate around the volune before rendering it. For the documentation, see [here](http://yt-project.org/docs/dev/visualizing/volume_rendering.html). For now, let's do something simple. We will:

1. Create a scene

2. Tell the camera how much to zoom in

3. Cut off a *little* bit of the light to reduce lens flair.

4. Show the plot.

yt has **way** more options, including very sophisticated options for the colors that are produced. The defaults are usually pretty good, but we're barely scratching the surface of what you can do.

In [ ]:
sc = yt.create_scene(ds,'density')
sc.camera.set_width(ds.arr(22, 'code_length'))
sc.show(sigma_clip=1)

### Exercise

See if you can use the scene interface to move around the spacetime and volume render from different angles.

## What Next?

By now you've hopefully gained some understanding of how to make plots in yt. There are many more plots to try if you're interested. In `~/data` you will find:

1. The Carpet output for the same static neutron star with a vertex centred refinement strategy

2. A `SimulationIO` file for the first two timesteps of a binary neutron star merger

3. A `SimulationIO` file for the full in-spiral and merger of two black holes. 

I've created skeletons notebooks for investigating these. See if you can make some visualizations using them!